# Solar physics with Python

Welcome to the solarnet Python tutorial!

## Tutorial 0: importing modules

If you want to use an external library in Python, you have to explicitly import it. It is generally best practice to import everything at the top of a file, so here we'll import all the libraries that we're going to use in this tutorial. Don't worry about what all of these are at the moment - as you go through the tutorial you'll learn what each one does.

In [ ]:
%matplotlib inline

import astropy.constants as const
import astropy.units as u
from astropy.visualization import quantity_support, time_support
# These commands allow us to plot astropy Quantities and Times
quantity_support()
time_support()

import matplotlib.pyplot as plt

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.time import parse_time

from heliopy.data import omni

## Tutorial 1: Searching and downloading data using sunpy

sunpy has a single interface for searching for and downloading data, called "Fido". For more in-depth information on searching for and downloading data see https://docs.sunpy.org/en/stable/guide/acquiring_data/index.html

Searching for data
---
Using the above defined search parameters, Fido can be used to search for data. It will automatically search several data sources, including the Virtual Solar Orbservatory (VSO), Joint Stanford Opererations Center (JSOC, for AIA/HMI/MDI data).

For more information on searching for and downloading data see https://docs.sunpy.org/en/stable/guide/acquiring_data/index.html

### Searching for data
In order to search for some data, you have to specify some search terms. In sunpy these are called "attributes". We have access to these as they were imported earlier using

`from sunpy.net import attrs as a`.

Examples of these are:
- `a.Time`
- `a.Instrument`
- `a.Wavelength`

For a full list, see https://docs.sunpy.org/en/stable/code_ref/net.html#classes

Lets have a go at doing a search:

In [ ]:
# Search for all observations within 10 minutes of this time
t = parse_time('2015-01-01 00:00:00')
dt = 10 * u.minute
result = Fido.search(a.Time(t, t + dt),
                     a.Instrument('AIA'),
                     a.Wavelength(193 * u.Angstrom))
result

You should see that the above search has returned 50 results. It's possible to download them all, but at ~10MB per image that would be 500MB to download. For now, we just want to download 1 image. To get a single result from here, we can 'slice' the result. This is just like slicing an array, but here:
- The first index indexes the provider (we only have one provider, the VSO here).
- The second index indexes the result table from that provider.


In this example we will get the 2nd result from the 1st provider.

In [ ]:
# Remember, python indexes from zero!
provider_idx = 0
result_idx = 1
single_result = result[provider_idx, result_idx]
# Or could have just done result[0, 1]
single_result

### Downloading files

Now we have a single result, we can use `Fido` to download it. This is done using `Fido.fetch(result)`, which returns a list of all the files that have been downloaded locally to your computer.

In [ ]:
downloaded_files = Fido.fetch(single_result)
print(downloaded_files)

Note that sunpy is clever: if you try to download a file with the same filename twice, it will notice you already have the file, and not try to download the file again.

In [ ]:
downloaded_files = Fido.fetch(single_result)

### Loading and plotting files
Now we have downloaded some data, we need to load it and plot it. ``sunpy.map.Map`` can be used to load any ``.fits`` file that contains a 2D map, creating a ``GenericMap`` object. We can then take a quick look at the image stored by calling ``map.peek()``:

In [ ]:
aiamap = sunpy.map.Map(downloaded_files[0])
aiamap.peek()

To get more detailed control of the plotting, we can create a Matplotlib `Figure` object and an `Axes` object that we then plot the map on to.

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(projection=aiamap)

image = aiamap.plot(ax)
aiamap.draw_grid(grid_spacing=20*u.deg)

ax.set_xlabel('Solar-X')
ax.set_ylabel('Solar-Y')

fig.colorbar(image)

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

Task 1:
    
By copying and changing code from above:
- Download and plot an AIA 193 image taken on your birthday in 2020
- Download and plot an AIA image in another wavelength (see the images on https://sdo.gsfc.nasa.gov for the other wavelengths available)

## Tutorial 3: Maps

In the previous section we had a quick look at how to download and plot maps. In this section, we'll have a quick look on how to access various bits of information about a Map.

A Map has two parts: the raw data array (`aiamap.data`), and metadata that allows you to interpret the raw data (`aiamap.meta`).

In [ ]:
aiamap.meta

This is a bit hard for a human to read! Luckily sunpy has a lot of helpful attributes that allow you to access the metadata. Lets have a quick look an example:

In [ ]:
# Print the shape of the map
aiamap.dimensions

<div style="background-color:#e6ffe6; padding:10px; border-style:
solid;; border-color:#00e600; border-width:1px">

Task 2:
    
Go to https://docs.sunpy.org/en/stable/api/sunpy.map.GenericMap.html, and scroll down to "Attributes". Find the right attributes to answer these questions:
- What is the resolution of the map (this is sometimes called the pixel scale)?
- How long was the exposure time?
- What was the Carrington latitude of the observer?

## Tutorial 4: Downloading in-situ data
We've had a good look at using sunpy to load and view remote sensing data. For in-situ data, the heliopy package (https://docs.heliopy.org/en/stable/) can be used to do similar things.

In particular, the ``heliopy.data`` module can be used to download and import a wide range of in situ datasets from various heliospheric missions, including (but not limited to!) Solar Orbiter, Parker Solar Probe, WIND, and ACE.

In this example we'll use data from the OMNI dataset. This is a combination of data from several spacecraft, and provides near continuous measurements of the solar wind at the orbit of the Earth for the last ~40 years.

We'll start by looking at solar wind that was around the same time as the aiamap we downloaded above:

In [ ]:
starttime = aiamap.date
endtime = aiamap.date + (25 * u.day)
omni_data = omni.h0_mrg1hr(starttime.datetime, endtime.datetime)
# Remove any gaps in the velocity data - this is important for later steps
omni_data._data = omni_data.to_dataframe().dropna(subset=['V'])

heliopy returns in-situ datasets as sunpy `TimeSeries` objects. These contain a time index (`data.index`), and a series of 1D data columns that can be accessed using `data.quantity(variable_name)`. Lets take a look at the columns:

In [ ]:
omni_data.columns

Whew, that's a lot of columns! Each one is documented at https://cdaweb.gsfc.nasa.gov/misc/NotesO.html#OMNI2_H0_MRG1HR in this case, but for now we're just interested in the N, V, T columns, which are the solar wind proton number density, speed, and temperature.

### Plotting in-situ data
Matplotlib can be used to plot the downloaded data. In this example we'll plot the solar wind speed, magnetic field mangitude, and number density.

In [ ]:
fig, axs = plt.subplots(figsize=(9, 6), nrows=3, sharex=True)

ax = axs[0]
ax.plot(omni_data.index, omni_data.quantity('V'), label='$v_{sw}$')
ax.set_title('Solar wind speed')

ax = axs[1]
ax.plot(omni_data.index, omni_data.quantity('ABS_B'), label='$|B|$')
ax.set_title('Magnetic field magnitude')

ax = axs[2]
ax.plot(omni_data.index, omni_data.quantity('N'), label='$n_{p}$')
ax.set_title('Proton number density')

## Tutorial 5: backmapping in-situ data

Now we have our solar wind data, measured at the Earth. We want to know what time it left the Sun, so we can line up features in the solar wind with features on the Sun. In general, the travel time is given by:

$$
t_{Sun} = t_{Earth} -  \int_{r_{Sun}}^{r_{Earth}} \frac{dr}{v_{sw} (r)} dr
$$

so we need to know what the solar wind speed was on the full trajectory from the Sun to the Earth. For now, lets just assume that the speed is constant, which gives us

$$
t_{Sun} =t_{Earth} - \frac{r_{Earth} - r_{Sun}}{v_{sw}}
$$

In [ ]:
def backmapped_time(t_earth, v_sw):
    t_earth = parse_time(t_earth)
    r_earth = 1 * const.au
    r_sun = const.R_sun
    return t_earth - (r_earth - r_sun) / vsw

t_sun = backmapped_time(omni_data.index, omni_data.quantity('V'))
# Calculate transit time
dt = (t_earth - t_sun).to(u.day)
dt

In [ ]:
# Plot the results
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(9, 6), tight_layout=True)
ax = axs[0]
ax.plot(t_earth.datetime, vsw)
ax.set_title('Solar wind speed')
ax.grid()

ax = axs[1]
ax.plot(t_sun.datetime, vsw)
ax.set_title('Solar wind speed')
ax.set_xlabel('Time (at the Sun)')
ax.grid()
ax.set_title('Backmapped solar wind speed')

ax.axvline(aiamap.date.datetime)
ax.text(aiamap.date.datetime, 550, ' AIA observation time')
ax.axvspan((aiamap.date - 1 * u.day).datetime,
           (aiamap.date + 1 * u.day).datetime,
           alpha=0.1)

fig.align_ylabels()
fig.autofmt_xdate()